# Emergency_Response_Incidents:

a. To get the required columns, use this module: 


1.   get_area_of_interest(df_spark, interested_columns)


b. Preprocessing pipeline: Pass your data through these functions. (if your columns fall in those categories)

1.   valid_date_check(date)
2.   valid_time_check(time)
3.   reverse_geo_code_boros(df_spark, Latitude, Longitude, Boro, lat_index, long_index)
4.   refine_age_group_race(df_spark, victim_age_group=None, suspect_age_group=None, suspect_race=None, victim_race=None)
5.   refine_sex_gender_impute(df_spark, suspect_age=None, suspect_gender=None, victim_age=None, victim_gender=None)
6.   refine_precinct_jur(df_spark, precinct=None, Jur_code=None)



In [1]:
!pip install pyspark
!pip install openclean

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 49.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=f0e25d3257e061dfa9f95f45fed41290050c9161f697c7fe84663e7724ebdfad
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 267 kB 35.5 MB/s 
     |████████████████████████████████| 69 kB 7.4 MB/s 
     |████████████████████████████████| 109 kB 57.3 MB/s 
     |████████████████████████████████| 137 kB 49.6 MB/s 
     |████████████████████████████████| 260 kB 64.6 MB/s 
     |████████████████████████████████| 596 kB 70.6 MB/s 
     |████████████████████████████████| 525 kB 71.4 MB/s 
     |████████████████████████████████| 180 kB 41.9 MB/s 
     |████████████████████████████████| 206 kB 71.2 MB/s 
     |████████████████████████████████| 149 kB 55.7 MB/s 
     |██████

In [2]:
#importing packages required
from pyspark import SparkContext, SparkConf
import os
import requests
from six.moves import urllib
import sys 
import pandas as pd
import matplotlib 
import matplotlib as plt
import numpy as np
import scipy as sp
import IPython
from IPython import display
import sklearn
import random
import time
import warnings
import re
import matplotlib.pyplot as plt
%matplotlib inline
from openclean.pipeline import stream
from openclean.profiling.column import DefaultColumnProfiler
from openclean.data.source.socrata import Socrata
from openclean.pipeline import stream
from openclean.function.eval.datatype import IsDatetime
import datetime
import pandas as pd
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import StringType

In [3]:
from geopy.geocoders import ArcGIS
geocoder=ArcGIS()
#example:
geocoder.reverse('40.61157006600007, -73.74736517199995')

Location(11-64 Redfern Ave, Far Rockaway, New York 11691, USA, (40.61161616586613, -73.74738361194636, 0.0))

In [4]:
#Creating Spark Session
sc = SparkContext.getOrCreate();
spark = SparkSession(sc)

In [5]:
#Downloading file from NYC Open Data
#https://drive.google.com/uc?export=download&id=1aeFR66MAgDXobrEosENZhBlq0dAqHEQw
#File uploaded on drive to make it available for all

fn_src = 'https://drive.google.com/uc?export=download&id=1aeFR66MAgDXobrEosENZhBlq0dAqHEQw'
fn_dst = '/content/Emergency_Response_Incidents.csv'

from six.moves import urllib

if os.path.isfile(fn_dst):
    print('File has already been downloaded', fn_dst)
else:
    print('Fetching file. This may take a while...', fn_dst)
    urllib.request.urlretrieve(fn_src, fn_dst)
    print('File %s has been downloaded' % fn_dst)

Fetching file. This may take a while... /content/Emergency_Response_Incidents.csv
File /content/Emergency_Response_Incidents.csv has been downloaded


In [6]:
src = 'https://data.beta.nyc/dataset/0ff93d2d-90ba-457c-9f7e-39e47bf2ac5f/resource/7caac650-d082-4aea-9f9b-3681d568e8a5/download/nyc_zip_borough_neighborhoods_pop.csv'
dst = 'nyc_zip_borough_neighborhoods_pop.csv'

#https://data.cityofnewyork.us/resource/h9gi-nx95.csv

from six.moves import urllib

if os.path.isfile(dst):
    print('File %s has already been downloaded' % dst)
else:
    urllib.request.urlretrieve(src, dst)
    print('File %s has been downloaded' % dst)

File nyc_zip_borough_neighborhoods_pop.csv has been downloaded


In [7]:
#similarly, lets get them into pyspark rdd
def get_area_of_interest(df_spark, interested_columns):
  df_spark=df_spark.select(interested_columns)
  return df_spark

# 2. Module for date related columns

As the dataset is for the data from 2006 to 2020, we can see that there is data from unknown format of "1010-05-14" to the year 2020. We need to clean this. Over here, we remove the null values where the complaint date is <2006. 

In [8]:
def valid_date_check(date, format):
  if date==None or date==" " or date=="":
      return False
  else:
    date_cpy=date
    split_date=date.split("-")
    format_date=format.split("-")
    if len(split_date)!=3 and len(format_date)!=3:
      date=date.split("/")
      format=format.split("/")
    else:
      date=split_date
      format=format_date
    if len(date)!=3:
      return False
    try:
      year=int(date[format.index('yyyy')])
      month=int(date[format.index('mm')])
      day=int(date[format.index('dd')])
      if year>=2006 and year<=2020:
        try:
          refined_date=datetime.datetime(year, month, day)
          return True
        except:
          return False
      else:
        return False
    except:
      return False

# 3. Module for time related columns

Similarly, lets check for the time as well. Here we must have time between 
the standard 24 hours.

In [9]:
#Deleting invalid time
def valid_time_check(time):
  if time==None or time==" " or time=="":
    return False
  else :
    cpy_time=time
    time=time.split(":")
    try:
      hour=int(time[0])
      mins=int(time[1])
      secs= int(time[2])
      # if hours is 24 then change it to 0 hours
      if hour == 24 and mins== 0 and secs == 0:
        hour=0
      try:
        newTime= datetime.time(hour,mins,secs)
        return True
      except :
        return False
    except:
      return False

#4. Module for Age Group and Race columns
The module works for only those columns whose column names are passed

In [10]:
def refine_age_group_race(df_spark, victim_age_group=None, suspect_age_group=None, suspect_race=None, victim_race=None):
  #params: dataframe, col names for the respective age, gender cols
  if victim_age_group:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_age_group])
  if suspect_age_group:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_age_group])
  if suspect_race:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_race])
  if victim_race:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_race])
  return df_spark

# 5. Module for Gender, Race Columns for suspects and victims

The module works for only those columns whose column names are passed

In [11]:
def refine_sex_gender_impute(df_spark, suspect_age=None, suspect_gender=None, victim_age=None, victim_gender=None):
  #params: dataframe, col names for the respective age, gender cols
  if suspect_age:
    df_spark=df_spark.na.fill("U",subset=[suspect_age])
  if victim_age:
    df_spark=df_spark.na.fill("U",subset=[victim_age])
  if suspect_gender:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_gender])
  if victim_gender:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_gender])
  return df_spark

# 6.a: Module for Precinct, Jurisdiction Code:
  dropping the null values

  The module works for only those columns whose column names are passed along with the df

In [12]:
def valid_precinct_check(precinct):
  if precinct==None or precinct==" " or precinct=="":
    return False
  else :
    return True

def valid_jur_check(jur):
  if jur==None or jur==" " or jur=="":
    return False
  else :
    return True

# 6.b Module for Reverse Geocoding the boroughs using latitudes and longitudes.

1. First we will remove the rows where latitude, longitude and boroughs are null. (around 450 tuples removed)
2. Then, where the boroughs are empty, take the latitude and longitude value and reverse geocode it using the module "reverseGeocoder".
3. Impute the borough name retrived in the empty space.


### USING MASTER DATASET
In the case of geocoding, geocoder gives us the zipcodes based on the latitude and longitude values. Inturn, we can use the master dataset of zipcodes inorder to retrive the borough names



NOTE: The dataset can be downloaded from : https://data.beta.nyc/en/dataset/pediacities-nyc-neighborhoods/resource/7caac650-d082-4aea-9f9b-3681d568e8a5

In [13]:
def reverse_geo_code_boros(df_spark, Latitude, Longitude, Boro, lat_index, long_index):
  #select data where we have to impute
  df_temp_boro_clean=df_spark.filter((df_spark[Latitude].isNotNull()) & (df_spark[Longitude].isNotNull()))
  boro_cleaner=df_temp_boro_clean.filter((df_temp_boro_clean[Boro].isNull())|(df_temp_boro_clean[Boro]=='NEW YORK'))

  # print("We have "+ str(boro_cleaner.count())+ " points to impute")
  print("___intializing Zip Code Look up ____")
  print("____ imputing the points ____")


  #use your path for master dataset here. 
  df_zips=pd.read_csv(dst)
  zip_master={}
  zips=df_zips['zip']
  boro=df_zips['borough']
  for i, j in zip(zips, boro):
    zip_master[i]=j
  zip_master[10020]='Manhattan'
  zip_master[11249]='Brooklyn'

  def reverseGeoCoder(latitude, longitude):
    loc=geocoder.reverse(str(latitude)+', '+str(longitude), timeout=1000)
    zipCode=str(loc).split(",")[2][-5:]
    if not int(zipCode) in zip_master:
      boro="UNKNOWN"
    else:
      boro=zip_master[int(zipCode)]
    boro=boro.upper()
    return boro

  #creating UD function
  ud_func= udf(reverseGeoCoder, StringType())
  boro_cleaned_dataframe = boro_cleaner.withColumn(Boro, ud_func(boro_cleaner[lat_index], boro_cleaner[long_index]))

  #joining the imputed dataset to the maindataset and returning

  joiner_dataset=df_spark.filter((df_spark[Latitude].isNotNull()) & (df_spark[Boro]!='NEW YORK') & (df_spark[Longitude].isNotNull()) & (df_spark[Boro].isNotNull()))
  fin_df=joiner_dataset.union(boro_cleaned_dataframe)
  return fin_df

NYPD Emergency_Response_Incidents 11085 Rows of data.

Confidence Level:	95% 
Confidence Interval: 5
Population:	
11000
      
Sample size needed:	
377


In [14]:
df_spark=spark.read.option("header",True).csv(fn_dst,inferSchema=True)
print(df_spark.count())
df_spark=df_spark.sample(0.03290)
df_spark.count()

11085


351

# PROFILING TO CHECK FOR NULL VALUES IN ALL THE COLUMNS

In [15]:
pandasDF = df_spark.toPandas()
ds=stream(pandasDF)
#Creating profile of our dataset
profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
Incident Type,351,0,98,0.279202,5.724212
Location,351,27,311,0.959877,8.237321
Borough,351,0,13,0.037037,2.432201
Creation Date,351,0,340,0.968661,8.382365
Closed Date,351,93,258,1.000000,8.011227
Latitude,351,53,277,0.929530,8.005247
Longitude,351,53,277,0.929530,8.005247


In [16]:
pandasDF.head()

,Incident Type,Location,Borough,Creation Date,Closed Date,Latitude,Longitude
0,Fire-2nd Alarm,238 East 24 Street,Manhattan,11/25/2016 04:06:09 AM,None,40.714422,-74.006076
1,Administration-Other,188 Street & 75 Avenue,Queens,05/10/2017 01:36:12 PM,None,40.714004,-73.829989
2,Utility-Gas Low Pressure,Noble Avenue & Watson Avenue,Bronx,01/30/2017 09:47:09 PM,None,40.827301,-73.869179
3,Fire-2nd Alarm,447 Wales Avenue,Bronx,07/26/2017 03:06:44 AM,None,40.810281,-73.908206
4,Fire-7th Alarm,110-14 Liberty Avenue,Queens,03/04/2017 11:13:52 PM,None,40.714004,-73.829989


## a. Select the columns that are common with the original dataset:
1. Incident Type
2. Borough
3. Creation Date
4. Latitude
5. Longitude



**We can consider the primary key along with this**
**"Incident Type"**


In [17]:
interested_columns_1=['Incident Type', 'Location', 'Borough', 'Creation Date', 'Latitude', 'Longitude']
df_spark=get_area_of_interest(df_spark, interested_columns_1)

In [18]:
df_spark.show()

+--------------------+--------------------+---------+--------------------+------------------+------------------+
|       Incident Type|            Location|  Borough|       Creation Date|          Latitude|         Longitude|
+--------------------+--------------------+---------+--------------------+------------------+------------------+
|      Fire-2nd Alarm|  238 East 24 Street|Manhattan|11/25/2016 04:06:...| 40.71442154062271|-74.00607638041981|
|Administration-Other|188 Street & 75 A...|   Queens|05/10/2017 01:36:...| 40.71400364095638|-73.82998933154158|
|Utility-Gas Low P...|Noble Avenue & Wa...|    Bronx|01/30/2017 09:47:...| 40.82730091310776|-73.86917897682753|
|      Fire-2nd Alarm|    447 Wales Avenue|    Bronx|07/26/2017 03:06:...| 40.81028097169481|-73.90820617557257|
|      Fire-7th Alarm|110-14 Liberty Av...|   Queens|03/04/2017 11:13:...| 40.71400364095638|-73.82998933154158|
|       Utility-Other|10 Nathan Perlman...|Manhattan|05/02/2019 11:30:...|40.733605005828394|-73

#RENAME COLUMNS which has "spaces" in COLUMN Names

In [19]:
from pyspark.sql.functions import upper, col
import pyspark.sql.functions as f
df_spark = df_spark.withColumnRenamed("Incident Type","Incident_Type")
df_spark = df_spark.withColumnRenamed("Creation Date","Creation_Date")
df_spark.tail(2)

[Row(Incident_Type='Fire-2nd Alarm', Location='549 Beach 68 Street', Borough='Far Rockaway', Creation_Date='10/29/2016 12:20:18 AM', Latitude=40.74862862455331, Longitude=-73.9856861527595),
 Row(Incident_Type='HazMat-Other', Location='East 70 Street & York Avenue', Borough='Manhattan', Creation_Date='11/03/2016 04:25:39 PM', Latitude=None, Longitude=None)]

#Converting datetime format to Date and Time
# Make sure to run this only once

In [20]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col, substring
df_spark = df_spark.select('*',substring(df_spark[3], 1, 10).alias('CREATE_DATE'))
df_spark = df_spark.select('*',substring(df_spark[3], 12, 8).alias('CREATE_TIME'))

In [21]:
df_spark = df_spark.drop('Creation_Date')
df_spark = df_spark.drop('Location') #as we have not used it in original dataset

In [22]:
df_spark.show()

+--------------------+---------+------------------+------------------+-----------+-----------+
|       Incident_Type|  Borough|          Latitude|         Longitude|CREATE_DATE|CREATE_TIME|
+--------------------+---------+------------------+------------------+-----------+-----------+
|      Fire-2nd Alarm|Manhattan| 40.71442154062271|-74.00607638041981| 11/25/2016|   04:06:09|
|Administration-Other|   Queens| 40.71400364095638|-73.82998933154158| 05/10/2017|   01:36:12|
|Utility-Gas Low P...|    Bronx| 40.82730091310776|-73.86917897682753| 01/30/2017|   09:47:09|
|      Fire-2nd Alarm|    Bronx| 40.81028097169481|-73.90820617557257| 07/26/2017|   03:06:44|
|      Fire-7th Alarm|   Queens| 40.71400364095638|-73.82998933154158| 03/04/2017|   11:13:52|
|       Utility-Other|Manhattan|40.733605005828394|-73.98293627906979| 05/02/2019|   11:30:49|
|      Fire-6th Alarm| Brooklyn| 40.64567498528483| -74.0022560985342| 04/03/2019|   05:10:29|
|        Fire-Haz Mat|   Queens| 40.71400364095638

## b. Lets pass the dataset through the preprocessing pipeline

In [23]:
df_temp=df_spark.rdd

1. Date and Time

### From data profiling we found that the date and time has no NULL Values

In [24]:
# checking date and time for null values

df_temp_=df_temp.map(lambda x:(x, valid_date_check(x[-2], 'mm/dd/yyyy'))).filter(lambda x: x[1]==True)
df_temp=df_temp_.map(lambda x: x[0])

In [25]:
df_temp_=df_temp.map(lambda x:(x, valid_time_check(str(x[-1])))).filter(lambda x: x[1]==True)
df_temp=df_temp_.map(lambda x: x[0])

3. Geocoding

In [26]:
#geospacial attributes imputation

df_temp=df_temp.toDF(schema=df_spark.schema)
df_spk=reverse_geo_code_boros(df_temp, 'Latitude', 'Longitude', 'Borough', -4, -3)

___intializing Zip Code Look up ____
____ imputing the points ____


Lets profile the data now.

In [27]:
pandasDF = df_spk.toPandas()
ds=stream(pandasDF)

#Creating profile of our dataset
profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
Incident_Type,273,0,82,0.300366,5.549748
Borough,273,0,12,0.043956,2.460617
Latitude,273,0,252,0.923077,7.859246
Longitude,273,0,252,0.923077,7.859246
CREATE_DATE,273,0,251,0.919414,7.912833
CREATE_TIME,273,0,263,0.963370,8.006275


#Precision and Recall

Selected data[Borough]  =351

Selected data[Lat]	= 351

Selected data[Lon]	= 351


---



True positive[borough] = 351 - 0 -78 = 273

True positive[lat]	= 351 -53 - 78 = 220

True positive[lat] 	= 351 - 53 - 78= 220


---



Relevant [borough] = 273 - 0 = 273

Relevant[latitude] = 273 - 53 = 220

Relevant[longitude] = 273 - 53 = 220


---



We are calculating precision as 

precision = True positive/selected

recall = True positive/Relavent


---


Precision[borough]:    87%

Precision[latitude]:  75%

Precision[longitude]:  75%


---



Recall[borough]:  99%

Recall[latitude]: 98%

Recall[longitude]:  99%


